In [1]:
import glob
import time
import os
import re
from osgeo import gdal
import warnings
from pyproj import Proj, transform
import osr
import numpy as np
import shutil as sh
from affine import Affine
import shutil
import geopandas
import rasterio
import utm
from shapely.geometry import Point
warnings.filterwarnings("ignore")

In [2]:
def save_multiband (output_name, dataset, raster_data, driver,
                    NaN_Value, nband, option, arr_projection=None):
    if arr_projection is None:
        arr_projection = []
    else:
        if str(type(arr_projection)) == "<class 'osgeo.gdal.Dataset'>":
            srs = arr_projection.GetProjectionRef()
        elif str(type(arr_projection)) == "<type 'int'>":
            srs = osr.SpatialReference()
            srs.ImportFromEPSG(arr_projection)
            srs = srs.ExportToPrettyWkt()
        else:
            srs = arr_projection
    NaN_rast = NaN_Value
    # Open the reference dataset
    g = dataset
    # Get the Geotransform vector
    if raster_data is False:
        raster_data = g.ReadAsArray()
        rast_arr = np.copy(raster_data)
    #auxiliar
    band = 0
    if type(raster_data) == tuple:
        rast_arr = np.array(raster_data[band])
    if str(type(g)) == "<class 'osgeo.gdal.Dataset'>":
        geo_transform = g.GetGeoTransform()
        x_size = g.RasterXSize  # Raster xsize
        y_size = g.RasterYSize  # Raster ysize
        srs = g.GetProjectionRef()  # Projection
    elif str(type(g)) == "<class 'affine.Affine'>":
        geo_transform = (g[2], g[0], g[1], g[5], g[3], g[4])
        rast_arr = raster_data[band,:,:]
        x_size = int(rast_arr.shape[1])
        y_size = int(rast_arr.shape[0])
    
    g = dataset
    driver = gdal.GetDriverByName("GTiff")
    dataset_out = driver.Create(output_name, x_size, y_size, nband,
                                gdal.GDT_Float32)
    #rast_arr[rast_arr == NaN_rast] = np.NaN
    geo_transform = (g[2], g[0], g[1], g[5], g[3], g[4])
    #dataset_out.SetGeoTransform(geo_transform)
    #dataset_out.SetProjection(srs)
    #end auxiliar
    for band in range(nband):
        if type(raster_data) == tuple:
            rast_arr = np.array(raster_data[band])
        if str(type(g)) == "<class 'osgeo.gdal.Dataset'>":
            geo_transform = g.GetGeoTransform()
            x_size = g.RasterXSize  # Raster xsize
            y_size = g.RasterYSize  # Raster ysize
            srs = g.GetProjectionRef()  # Projection
        elif str(type(g)) == "<class 'affine.Affine'>":
            geo_transform = (g[2], g[0], g[1], g[5], g[3], g[4])
            rast_arr = raster_data[band,:,:]
            x_size = int(rast_arr.shape[1])
            y_size = int(rast_arr.shape[0])
        #PROCESS RASTERIO NUMPY
        else:
            geo_transform = (g[1][2], g[1][0], g[1][1], g[1][5], g[1][3], g[1][4])
            rast_arr = np.array(g[0])
            x_size = int(rast_arr.shape[2])
            y_size = int(rast_arr.shape[1])

        if option==1:
            rast_arr[rast_arr == NaN_rast] = np.NaN
            dataset_out.SetGeoTransform(geo_transform)
            dataset_out.SetProjection(srs)
            dataset_out.GetRasterBand(band + 1).WriteArray(
                rast_arr.astype(np.float32))
        elif option==2:
            dataset_out.SetGeoTransform(geo_transform)
            dataset_out.SetProjection(srs)
            dataset_out.GetRasterBand(band + 1).WriteArray(
                rast_arr.astype(np.float32))
            
#         rast_arr[rast_arr == NaN_rast] = np.NaN     
#         dataset_out.SetGeoTransform(geo_transform)
#         dataset_out.SetProjection(srs)
#         dataset_out.GetRasterBand(band + 1).WriteArray(rast_arr.astype(np.float32))

In [3]:
def GetTransform(Transform):
    transform = Transform.GetGeoTransform()
    Aff = Affine(transform[1], transform[2], transform[0], transform[4],
                 transform[5], transform[3])
    return(Aff)

In [4]:
#Get directory with all raw sat images for Rayleigh atmospheric correction
path_to_s2 = r'/mnt/data4/antonis/dataset_new'
s2_files = glob.glob(path_to_s2 + "/*")

In [5]:
#Step1: Specify product name to be ACOLITE corrected

In [31]:
requested_product = 'S2B_MSIL1C_20180712T071619_N0206_R006_T39RUM_20180712T110801'
for k in range(len(s2_files)):
    if requested_product in s2_files[k][-60:]:
        print('%s is present in the list' % requested_product)
        requested_path = s2_files[0][:-60] + requested_product

S2B_MSIL1C_20180712T071619_N0206_R006_T39RUM_20180712T110801 is present in the list


In [32]:
#Step2: Show path of RAW data folder to ACOLITE correction
requested_path+'/RAW/'+requested_product+'.SAFE'

'/mnt/data4/antonis/dataset_new/S2B_MSIL1C_20180712T071619_N0206_R006_T39RUM_20180712T110801/RAW/S2B_MSIL1C_20180712T071619_N0206_R006_T39RUM_20180712T110801.SAFE'

In [33]:
#Step3: Run ACOLITE
#Step 1:
print('Step1: Copying settings_file from acolite to image folder')
#Copy settings_file from acolite folder to image folder to proceed atmospherical corrections
src = r'/home/antonis/acolite_py_linux/dist/acolite/settings_file.txt'
dst =r'/home/antonis/sentinel-2/'
sh.copy(src, dst)

#Step2:
print('Step2: Creation of folder of the desired atmosphericly corrected product')
#Create folder of the desired atmosphericly corrected product
output_folder = r'/mnt/data4/antonis/acolite_corrected' +str('/') + requested_product +'.SAFE'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    print ("Successfully created the directory %s " % output_folder)
else:
    print ("Directory %s already exists" % output_folder)

print('Now processing: ',requested_product)

#Step3:
print('Step3: Change inputs inside settings file (inputfile, outputfile, extend)')
#Change inputs inside settings file (inputfile, outputfile, extend)
#Currently parameter specific based on the original settings_file located in the acolite folder
file = r'/home/antonis/sentinel-2/settings_file.txt'
ouput = r'/home/antonis/sentinel-2/settings_file_n.txt'
output_dir = output_folder

with open(file, 'r') as f:
    with open(ouput, 'wt') as new_f:
        for line in f:
            new_line = line.replace(r'inputfile=C:\Users\antonkout\Documents\Work\NTUA\Data\Preprocess\LE07_L2SP_021040_20100501_20200911_02_T1', 'inputfile='+requested_path+'/RAW/'+requested_product+'.SAFE')
            new_f.write(new_line)

file = r'/home/antonis/sentinel-2/settings_file_n.txt'
ouput = r'/home/antonis/sentinel-2/settings_file.txt'

with open(file, 'r') as f:
    with open(ouput, 'wt') as new_f:
        for line in f:
            new_line = line.replace(r'output=C:\Users\antonkout\Documents\Work\NTUA\Data\Preprocess\LE07_L2SP_021040_20100501_20200911_02_T1_results', 'output='+output_dir)
            new_f.write(new_line)

if os.path.exists(file): 
    os.remove(file)
else: 
    print("The file does not exist") 

#To find extend must first find the desired sat img
for root, dirs, files in os.walk(requested_path+'/RAW/'+requested_product+'.SAFE'):
    for name in files:
        if name.endswith("_B01.jp2"):
            sat = root+str(r'/')+name
            print(sat)
#Step4
print('Step4: Update extend parameter in settings_file using gdal')
#Get extend using gdal
ds = gdal.Open(sat, 1)
geotransform = ds.GetGeoTransform()
proj = osr.SpatialReference(wkt=ds.GetProjection())
inprj = 'epsg:'+proj.GetAttrValue('AUTHORITY',1)

upx, xres, xskew, upy, yskew, yres = geotransform
cols = ds.RasterXSize
rows = ds.RasterYSize

ulx = upx + 0*xres + 0*xskew
uly = upy + 0*yskew + 0*yres

llx = upx + 0*xres + rows*xskew
lly = upy + 0*yskew + rows*yres

lrx = upx + cols*xres + rows*xskew
lry = upy + cols*yskew + rows*yres

urx = upx + cols*xres + 0*xskew
ury = upy + cols*yskew + 0*yres

inProj = Proj(init=inprj)
outProj = Proj(init='epsg:4326')

#Transform coordinated to desired epsg
ext = (ulx,uly),(llx,lly),(lrx,lry),(urx,ury)
extend=[]
for k in range(len(ext)):
    extend.append(transform(inProj,outProj,ext[k][0],ext[k][1]))
xmax,xmin,ymax,ymin = np.asarray(extend)[:,1].max(),np.asarray(extend)[:,1].min(),np.asarray(extend)[:,0].max(),np.asarray(extend)[:,0].min()
extend = xmin,ymin,xmax,ymax
extend = str(extend).replace('(','').replace(')','')

#Replace with the new extend
file =r'/home/antonis/sentinel-2/settings_file.txt'
ouput = r'/home/antonis/sentinel-2/settings_file_n.txt'

with open(file, 'r') as f:
    with open(ouput, 'wt') as new_f:
        for line in f:
            new_line = line.replace(r'limit= 27.883946,  -90.264583,  29.858269, -87.733565', 'limit= '+extend)
            new_f.write(new_line)

#Rename text file to the origin name and remove the temp one
if os.path.exists(file): 
    os.remove(file)
else: 
    print("The file does not exist") 
os.rename(ouput,file)

#Step 5:
print('Step5: Run ACOLITE based on the updated settings_file')
#Run ACOLITE based on the constructed settings_file
starttime=time.time()
cmd = r'/home/antonis/acolite_py_linux/dist/acolite/acolite --cli --settings='+'"{}"'.format(file)
print ("Running...", cmd)
os.system(cmd)
elapsedtime=time.time()-starttime
mins, secs = divmod(elapsedtime, 60)
hours, mins = divmod(mins, 60)
print ("Acolite algorithm completed for:", requested_product, "in: {}:{}:{}".format(int(hours), int(mins), round(secs,2)))

#Remove settings_file in order to copy it again from the acolite installed folder
#os.remove(file)

Step1: Copying settings_file from acolite to image folder
Step2: Creation of folder of the desired atmosphericly corrected product
Successfully created the directory /mnt/data4/antonis/acolite_corrected/S2B_MSIL1C_20180712T071619_N0206_R006_T39RUM_20180712T110801.SAFE 
Now processing:  S2B_MSIL1C_20180712T071619_N0206_R006_T39RUM_20180712T110801
Step3: Change inputs inside settings file (inputfile, outputfile, extend)
/mnt/data4/antonis/dataset_new/S2B_MSIL1C_20180712T071619_N0206_R006_T39RUM_20180712T110801/RAW/S2B_MSIL1C_20180712T071619_N0206_R006_T39RUM_20180712T110801.SAFE/GRANULE/L1C_T39RUM_A007039_20180712T072748/IMG_DATA/T39RUM_20180712T071619_B01.jp2
Step4: Update extend parameter in settings_file using gdal
Step5: Run ACOLITE based on the updated settings_file
Running... /home/antonis/acolite_py_linux/dist/acolite/acolite --cli --settings="/home/antonis/sentinel-2/settings_file.txt"
Acolite algorithm completed for: S2B_MSIL1C_20180712T071619_N0206_R006_T39RUM_20180712T110801 i

In [34]:
#Step4: Create multiband rhorc
sat = []
indx = ['_443.tif', '_492.tif', '_560.tif', '_665.tif', '_704.tif', '_740.tif', '_783.tif', '_833.tif', '_865.tif', '_1614.tif', '_2202.tif']
indxn = ['_442.tif', '_492.tif', '_559.tif', '_665.tif', '_704.tif', '_739.tif', '_780.tif', '_833.tif', '_864.tif', '_1610.tif', '_2186.tif']

#To find extend must first find the desired sat img
for root, dirs, files in os.walk(output_folder):
    for name in files:
        if 'rhorc' in name:
            sat.append(root+str(r'/')+name)
rhorc_sat=[]
for a in range(len(sat)):
    if any(sat[a][-8:] in s for s in indx):
        flag = 1
        rhorc_sat.append(sat[a])

if len(rhorc_sat)<10:
    rhorc_sat = []
    flag = 2 
    for a in range(len(sat)):
        if any(sat[a][-8:] in s for s in indxn):
            rhorc_sat.append(sat[a])

rhorc_index =[]
for k in range(len(rhorc_sat)):
    if flag == 1:
        rhorc_index.append(rhorc_sat[k][:143]+indx[k])
    elif flag == 2:
        rhorc_index.append(rhorc_sat[k][:143]+indxn[k])
    
B1, B2, B3, B4, B5, B6, B7, B8, B9, B10, B11 = gdal.Open(rhorc_index[0]), gdal.Open(rhorc_index[1]), gdal.Open(rhorc_index[2]), gdal.Open(rhorc_index[3]),gdal.Open(rhorc_index[4]), gdal.Open(rhorc_index[5]), gdal.Open(rhorc_index[6]), gdal.Open(rhorc_index[7]), gdal.Open(rhorc_index[8]), gdal.Open(rhorc_index[9]), gdal.Open(rhorc_index[10])
srs = B1.GetProjectionRef()
b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11 = (B1.ReadAsArray(), B2.ReadAsArray(),B3.ReadAsArray(), B4.ReadAsArray(),
                                                  B5.ReadAsArray(), B6.ReadAsArray(),B7.ReadAsArray(), B8.ReadAsArray(),
                                                  B9.ReadAsArray(), B10.ReadAsArray(), B11.ReadAsArray())
B = np.stack([b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11])
for k in range(B.shape[0]):
    B[k] = np.where(B[k]==9.96921e+36, 0, B[k])
    
rhorc_file = requested_path+'/ACOLITE/'+requested_product+'_rhorc.tif'
option = 1
save_multiband(rhorc_file, GetTransform(B1), B, "GTiff", 0, B.shape[0],option, srs)

In [35]:
#Step5: Delete folder of acolite corrected
shutil.rmtree(output_folder)

In [36]:
#Step6: Create RGB composite
sat = []
indx = ['_B02.jp2', '_B03.jp2', '_B04.jp2']
#To find extend must first find the desired sat img
for root, dirs, files in os.walk(requested_path+'/RAW/'+requested_product+'.SAFE'):
    for name in files:
        if '.jp2' in name:
            sat.append(root+str(r'/')+name)
raw_sat=[]
for a in range(len(sat)):
    if any(sat[a][-8:] in s for s in indx):
        raw_sat.append(sat[a])

rgb_index =[]
for k in range(len(raw_sat)):
    rgb_index.append(raw_sat[k][:-8]+indx[k])
    
B2, B3, B4 = gdal.Open(rgb_index[0]), gdal.Open(rgb_index[1]), gdal.Open(rgb_index[2])
srs = B2.GetProjectionRef()
b2, b3, b4 = (B2.ReadAsArray(), B3.ReadAsArray(),B4.ReadAsArray())
B = np.stack([b2, b3, b4])

rgb_file = requested_path+'/RGB/'+requested_product+'_rgb.tif'
option = 2
save_multiband(rgb_file, GetTransform(B2), B, "GTiff", 0, B.shape[0], option, srs)